In [408]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [296]:
full_data = pd.read_csv('Full Data.txt', index_col = [0])

In [297]:
full_data

,gender,ajcc_pathologic_tumor_stage,tumor_status,DSS,DSS.time,TP53,TTN,FAT1,MUC16,CSMD3,...,NIPBL,CEACAM5,CEACAM6,CLCA4,EGFR,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,Splice_Site,age
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-4P-AA8J,1,0,0,0.0,0.279452,822.678,1157.3300,1620.71,1004.4700,1.2325,...,1595.440,56.0776,423.3550,502.2340,722.847,92,8,0,2,1
TCGA-BA-4074,1,0,1,1.0,1.265753,1805.510,103.6870,6415.45,17.3422,0.3854,...,1605.890,153.3820,275.9330,12.7176,1267.900,106,8,0,1,1
TCGA-BA-4075,1,1,1,1.0,0.775342,383.760,295.9180,4162.40,1.0720,36.9824,...,907.410,18.2232,45.5581,10.1836,4833.430,113,10,0,4,0
TCGA-BA-4077,0,0,1,1.0,3.106849,1929.260,207.3870,7246.87,432.6270,0.0000,...,1174.690,2392.6900,2437.9200,720.8850,2371.940,234,20,0,6,0
TCGA-BA-5149,1,0,1,1.0,2.208219,165.298,347.9540,8848.73,110.5410,0.3422,...,1798.770,37.3032,447.6390,5.4757,6934.280,96,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-UF-A7JO,0,0,1,1.0,1.728767,1604.880,176.1930,1881.36,347.7040,2.8008,...,854.256,1498.8500,2014.6000,82.8248,530.911,135,9,0,4,1
TCGA-UF-A7JS,1,0,1,1.0,1.863014,2457.400,1379.4300,1339.31,243.6470,19.4320,...,982.063,358.7440,294.4690,32.8849,246.637,50,1,0,1,1
TCGA-UF-A7JT,0,0,1,1.0,2.720548,1684.850,149.5190,3430.74,213.8530,14.2857,...,1009.520,49.7835,209.5240,200.4330,790.905,152,10,0,3,1


# Simple Columns

In [378]:
X = df.drop(columns = ['IA_SKIP'])
y = df['IA_SKIP']
X = sm.add_constant(X)
model = sm.Logit(y, X)
result = model.fit()

# Get the pvalues of the model
p_values = result.pvalues
p_values = p_values[p_values < 0.05]
p_values

# Log-rank test

In [393]:
def quantile_gene(df, x):
    q1 = df[x].quantile(0.1)
    q3 = df[x].quantile(0.9)
    df[x + 'binary'] = df[x].apply(lambda x: 0 if x <= q1 else 1 if x >= q3 else np.nan)

In [394]:
def logrank(df, lis, time):
    dic ={}
    data = df[df['DSS.time'] <= time]
    for i in lis:
        quantile_gene(data, i)
        low_expression = data[data[i + 'binary'] == 0]
        high_expression =  data[data[i +'binary'] == 1]
        # logrank
        results = logrank_test(high_expression['DSS.time'], low_expression['DSS.time'],
                       high_expression['DSS'], low_expression['DSS'])
        z_value = results.test_statistic
        if z_value > 1.96 or z_value < -1.96:
            dic[i] = z_value
    
    # return a dictionary of name and z value
    return dic

In [395]:
gene_expression = list(full_data.drop(columns = ['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status', 'DSS' ,
                                      'DSS.time', 'Missense_Mutation', 'Nonsense_Mutation', 
                                      'Nonstop_Mutation', 'Splice_Site', 'age']).columns)

In [396]:
# for gene expression
a = logrank(full_data,gene_expression, 2) # example: use full data and 2 years as limit

In [400]:
time_limit = [1, 2, 5, 7, 10]
feature_dic = {}
for i in time_limit:
    dic = logrank(full_data, gene_expression, i)
    feature_dic[i] = list(dic.keys())

In [302]:
#select top n genes by sort z values as features from full_data

In [303]:
mutation = ['Missense_Mutation', 'Nonsense_Mutation', 'Nonstop_Mutation', 'Splice_Site']

In [304]:
# for mutation
logrank(full_data,mutation, 2) # example: use full data and 2 years as limit

{'Missense_Mutation': 2.2088500784074974,
 'Nonsense_Mutation': 1.9839793008189441,
 'Splice_Site': 5.71020812335661}

# Cox model

In [308]:
def coxfitter(data, duration, event):
    # Define the CoxPHFitter object
    cph = CoxPHFitter()
    # Fit the model to your data
    cph.fit(data, duration_col= duration, event_col=event)
    summary = cph.summary
    return summary

In [315]:
def cox_model(df, time, lis):
    data = df[df['DSS.time'] <= time]
    signi = []
    pvals = []
    pvals_correct = []
    for i in lis:
        summary = coxfitter(data[[i ,'gender','ajcc_pathologic_tumor_stage', 
                                   'tumor_status', 'age', 'DSS.time', 'DSS']], 'DSS.time', 'DSS')
        p_values = summary["p"]
        rejected, pvals_corrected = multipletests(p_values, method='fdr_bh')[:2]
        summary['p_corrected'] = pvals_corrected
        p = summary.loc[i, "p"]
        p_correct = summary.loc[i, "p_corrected"]
        pvals.append(p)
        pvals_correct.append(p_correct)
        signi.append(rejected[0])
    features = [col for col in lis]  # list of features
    # Create a DataFrame with the results
    results = pd.DataFrame({'Features': features,'p': pvals,'p_corrected': pvals_correct, 'significant': signi})
    return results[results['significant'] == True]


In [443]:
# gene expression level
time_limit = range(1,11)
feature_dic = {}
for i in time_limit:
    df = cox_model(full_data, i, gene_expression)
    feature_dic[i] = list(df['Features'])
feature_dic

{1: ['DCHS2'],
 2: ['CASP8', 'DNAH9', 'DNAH11', 'LRP2', 'RP1', 'DNAH7', 'MDN1'],
 3: ['MUC16', 'DNAH9', 'RP1', 'DNAH7'],
 4: ['MUC16', 'DNAH9', 'RP1', 'DNAH7'],
 5: ['MUC16', 'DNAH9', 'DNAH11', 'RP1', 'DNAH7'],
 6: ['MUC16', 'DNAH9', 'DNAH11', 'RP1', 'DNAH7'],
 7: ['MUC16', 'DNAH9', 'DNAH11', 'RP1', 'DNAH7'],
 8: ['MUC16', 'DNAH9', 'DNAH11', 'RP1', 'DNAH7'],
 9: ['MUC16', 'DNAH9', 'DNAH11', 'RP1', 'DNAH7'],
 10: ['MUC16', 'DNAH9', 'DNAH11', 'RP1', 'DNAH7']}

In [437]:
# mutation
cox_model(full_data, 2, mutation)# example: use full data and 2 years as limit

,Features,p,p_corrected,significant
0,Missense_Mutation,4.747304e-08,2.373652e-07,True
1,Nonsense_Mutation,3.141682e-04,1.570841e-03,True
2,Nonstop_Mutation,8.020092e-03,4.010046e-02,True
3,Splice_Site,3.520589e-05,1.760294e-04,True


# Classification

In [318]:
#select top n mutations by z values as features columns from full_data

In [369]:
def classifcation_svm(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # SVM
   
    # Define the model
    svm_clf = svm.SVC()

    # Train the model
    svm_clf.fit(X_train, y_train)
   
    # Predict on the test set
    svm_predictions = svm_clf.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, svm_predictions)
    #print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

In [375]:
for i in time_limit:
    lis = []
    for k in range(10000):
        accuracy = classifcation_svm(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and gene expression features from Cox model')
    print('SVM Accuracy ' + str(mean_acc))

For time limit 1 and gene expression features from Cox model
SVM Accuracy 0.7046266666666587
For time limit 2 and gene expression features from Cox model
SVM Accuracy 0.7070577777777708
For time limit 5 and gene expression features from Cox model
SVM Accuracy 0.7090733333333264
For time limit 7 and gene expression features from Cox model
SVM Accuracy 0.708897777777772
For time limit 10 and gene expression features from Cox model
SVM Accuracy 0.7092722222222169


In [444]:
def classifcation_randomforest(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    rf_clf = RandomForestClassifier()

    # Train the model
    rf_clf.fit(X_train, y_train)
    # Predict on the test set
    rf_predictions = rf_clf.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, rf_predictions)
    #print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))

In [367]:
for i in time_limit:
    lis = []
    for k in range(10000):
        accuracy = classifcation_randomforest(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and gene expression features from Cox model')
    print('Random Forest Accuracy ' + str(mean_acc))

For time limit 1 and gene expression features from Cox model
Random Forest Accuracy: 0.5111111111111111
For time limit 2 and gene expression features from Cox model
Random Forest Accuracy: 0.5777777777777777
For time limit 5 and gene expression features from Cox model
Random Forest Accuracy: 0.7222222222222222
For time limit 7 and gene expression features from Cox model
Random Forest Accuracy: 0.6222222222222222
For time limit 10 and gene expression features from Cox model
Random Forest Accuracy: 0.6333333333333333


In [445]:
def best_estimator(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    rf_clf = RandomForestClassifier()
    param_grid = { 
    'n_estimators': [10, 50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']}
    # Train the model
    CV_rf = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv = 5)
    CV_rf.fit(X_train, y_train)
    # Predict on the test set
    return CV_rf.best_params_

In [446]:
# get the best parameter for each time based on each feature
para_dic = {}
for i in time_limit:
    para_dic[i] = best_estimator(full_data, feature_dic[i])

In [440]:
def classifcation_randomforestcv(df, features, best_para):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    rf_best = RandomForestClassifier(n_estimators= best_para['n_estimators'], max_features=best_para['max_features'], max_depth=best_para['max_depth'], criterion=best_para['criterion'])
    rf_best.fit(X_train, y_train)

    predictions = rf_best.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    cv_score = cross_val_score(rf_best, X, y, cv=5)
    # Evaluate the model
    return accuracy, cv_score.mean()

In [448]:
for i in time_limit:
    lis_grid = []
    lis_cv = []
    for k in range(10):
        grid_accuracy, cv_score = classifcation_randomforestcv(full_data, feature_dic[i],para_dic[i])
        lis_grid.append(grid_accuracy)
        lis_cv.append(cv_score)
    mean_grid = sum(lis_grid)/len(lis_grid)
    mean_cv =  sum(lis_cv)/len(lis_cv)
    print('For time limit ' + str(i) + ' and gene expression features from Cox model')
    print('Random Forest GridSearchCV Accuracy ' + str(mean_grid))
    print('Random Forest cross validation Accuracy ' + str(mean_cv))

For time limit 1 and gene expression features from Cox model
Random Forest GridSearchCV Accuracy 0.6900000000000002
Random Forest cross validation Accuracy 0.6844444444444445
For time limit 2 and gene expression features from Cox model
Random Forest GridSearchCV Accuracy 0.6866666666666668
Random Forest cross validation Accuracy 0.6973333333333334
For time limit 3 and gene expression features from Cox model
Random Forest GridSearchCV Accuracy 0.6944444444444445
Random Forest cross validation Accuracy 0.6908888888888889
For time limit 4 and gene expression features from Cox model
Random Forest GridSearchCV Accuracy 0.7144444444444444
Random Forest cross validation Accuracy 0.6906666666666667
For time limit 5 and gene expression features from Cox model
Random Forest GridSearchCV Accuracy 0.668888888888889
Random Forest cross validation Accuracy 0.6706666666666667
For time limit 6 and gene expression features from Cox model
Random Forest GridSearchCV Accuracy 0.7100000000000001
Random For

In [378]:
def classifcation_logistic(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    log_reg = LogisticRegression()

    # Train the model
    log_reg.fit(X_train, y_train)
    log_reg_predictions = log_reg.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, log_reg_predictions)
    #print("Logistic Regression Accuracy:", accuracy_score(y_test, log_reg_predictions))

In [379]:
for i in time_limit:
    lis = []
    for k in range(10000):
        accuracy = classifcation_logistic(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and gene expression features from Cox model')
    print('Logistic Regression Accuracy ' + str(mean_acc))

For time limit 1 and gene expression features from Cox model
Logistic Regression Accuracy 0.704196666666658
For time limit 2 and gene expression features from Cox model
Logistic Regression Accuracy 0.7024999999999927
For time limit 5 and gene expression features from Cox model
Logistic Regression Accuracy 0.703144444444442
For time limit 7 and gene expression features from Cox model
Logistic Regression Accuracy 0.7031322222222158
For time limit 10 and gene expression features from Cox model
Logistic Regression Accuracy 0.7030522222222149


In [406]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index

y = full_data[['DSS']].values  # Convert to NumPy array
X = full_data[['gender','TP53','TTN','FAT1','MUC16','CLCA4','EGFR','Missense_Mutation','Nonsense_Mutation','Nonstop_Mutation','Splice_Site','age']].values  # Convert to NumPy array
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
# Define the custom loss function for survival analysis
def cox_loss(y_pred, y):
    hazard_ratio = np.exp(y_pred)
    log_risk = np.log(np.cumsum(hazard_ratio))
    log_lik = y_pred - log_risk
    neg_log_lik = -np.sum(log_lik * y - log_risk * np.sum(y))
    return neg_log_lik
# Initialize the XGBoost regressor
model = xgb.XGBRegressor(objective=cox_loss)
# Train the model
model.fit(X_train, y_train)
# Make predictions on the test set
y_pred = model.predict(X_test)
# Calculate the concordance index (C-index)
cindex = concordance_index(y_test, -y_pred, np.squeeze(y_test))
print("Concordance Index (C-index):", cindex)

TypeError: cannot unpack non-iterable numpy.float32 object

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lifelines import CoxPHFitter
from keras.models import Sequential
from keras.layers import Dense
# Load and preprocess the dataset
dataset = pd.read_csv('Full Data.txt')
#X = dataset.drop(['DSS.time', 'DSS'], axis=1)
X = dataset[['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status','TP53','TTN','FAT1','MUC16','CLCA4','EGFR','Missense_Mutation',	'Nonsense_Mutation',	'Nonstop_Mutation',	'Splice_Site',	'age']]  # Replace with the actual feature columns
y = dataset[['DSS.time', 'DSS']]
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Define and train the feedforward neural network
input_dim = X_train_scaled.shape[1]
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=input_dim))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train_scaled, y_train['DSS.time'], epochs=10, batch_size=32)
# Extract the learned representations from the hidden layers
hidden_layer_outputs = model.predict(X_train_scaled)
# Combine the learned representations with the survival data
X_train_combined = np.concatenate((X_train_scaled, hidden_layer_outputs), axis=1)
# Fit the Cox proportional hazards model
cox_nnet = CoxPHFitter()
cox_nnet.fit(pd.DataFrame(X_train_combined), duration_col=dataset[['DSS.time']], event_col=dataset[['DSS']])
# Predict survival probabilities for the test set
hidden_layer_outputs_test = model.predict(X_test_scaled)
X_test_combined = np.concatenate((X_test_scaled, hidden_layer_outputs_test), axis=1)
survival_probs = cox_nnet.predict_survival_function(pd.DataFrame(X_test_combined))
# Evaluate the model
c_index = cox_nnet.score(pd.DataFrame(X_test_combined), y_test, scoring_method="concordance_index")
print("Concordance Index: ", c_index)
11:36
This is the error: ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().